In [ ]:
import torch
import numpy as np
from scipy.special import softmax
from scipy.optimize import minimize
from sklearn.metrics import log_loss

## Expected Calibration Error (ECE)

In [ ]:
def one_hot_encode(labels, num_classes=None):
  if num_classes is None:
    num_classes = len(np.unique(labels))
  return np.eye(num_classes)[labels]


def get_calibration_error(probs, labels, bin_upper_bounds, num_bins):
    if np.size(probs) == 0:
        return 0

    bin_indices = np.digitize(probs, bin_upper_bounds)
    sums = np.bincount(bin_indices, weights=probs, minlength=num_bins)
    sums = sums.astype(np.float64)
    counts = np.bincount(bin_indices, minlength=num_bins)
    counts = counts + np.finfo(sums.dtype).eps
    confs = sums / counts
    accs = np.bincount(bin_indices, weights=labels, minlength=num_bins) / counts

    calibration_errors = accs - confs

    weighting = counts / float(len(probs.flatten()))
    weighted_calibration_error = calibration_errors * weighting

    return np.sum(np.abs(weighted_calibration_error))

def ECE(probs, labels, num_bins=10):
    num_classes = probs.shape[1]
    labels_matrix = one_hot_encode(labels, probs.shape[1])

    bin_upper_bounds = np.histogram_bin_edges([], bins=num_bins, range=(0.0, 1.0))[1:]

    labels_matrix = labels_matrix[range(len(probs)), np.argmax(probs, axis=1)]
    probs_matrix = probs[range(len(probs)), np.argmax(probs, axis=1)]

    calibration_error = get_calibration_error(probs_matrix.flatten(), labels_matrix.flatten(), bin_upper_bounds, num_bins)

    return calibration_error

## Brier Score (BS)

In [ ]:
def BS(probs, labels):
    n_samples, n_classes = probs.shape
    labels_matrix = one_hot_encode(labels, n_classes)
    brier_score = np.sum((probs - labels_matrix) ** 2) / n_samples

    return brier_score

## Softmax score

In [ ]:
def softmax_score(outputs):
    return softmax(outputs, axis=1)

# input the outputs from the label
test_logits = np.array([[1.0, 2.0, 3.0],
                          [1.0, 2.0, 0.5]])

test_probs = softmax_score(test_logits)
print(test_probs)

test_labels = np.array([0, 1])

ece = ECE(test_probs, test_labels)
bs = BS(test_probs, test_labels)

print(f'ECE: {ece:.4f}')
print(f'BS: {bs:.4f}')

[[0.09003057 0.24472847 0.66524096]
 [0.2312239  0.62853172 0.14024438]]
ECE: 0.1469
BS: 0.7708


## Deep ensemble

In [ ]:
def get_outputs(model, data_loader, device='cuda'):
    model.eval()
    all_labels, all_logits = [], []

    with torch.no_grad():
        for idx, (images, labels, _) in enumerate(data_loader):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            all_labels.append(labels.detach().cpu().numpy())
            all_logits.append(outputs.detach().cpu().numpy())

    all_labels = np.concatenate(all_labels, axis=0)
    all_logits = np.concatenate(all_logits, axis=0)

    return all_labels, all_logits


def evaluate_ensemble(test_loader, num_ensemble=10):
    test_probs = [], []
    for i in range(num_ensemble):
        # load a sub model
        model = ...

        test_labels, test_logits = get_outputs(model, test_loader)
        test_probs.append(softmax(test_logits, axis=1))

    test_probs = np.array(test_probs)
    test_probs_mean, test_probs_std = test_probs.mean(0), test_probs.std(0)

    ece_de = ECE(test_probs_mean, test_labels)
    bs_de = BS(test_probs_mean, test_labels)

# train a set of sub models

## MC dropout

In [ ]:
def evaluate_dropout(model, test_loader, forward_passes=10, device='cuda'):
    def enable_dropout(model):
        """ Function to enable the dropout layers during test-time """
        for m in model.modules():
            if m.__class__.__name__.startswith('Dropout'):
                m.train()

    def get_outputs_dropout(model, data_loader):
        all_labels, all_logits = [], []
        model.eval()
        enable_dropout(model)
        with torch.no_grad():
            for idx, (images, labels, _) in enumerate(data_loader):
                images, labels = images.to(device), labels.to(device)
                outputs = model(images, dropout=True)

                all_labels.append(labels.detach().cpu().numpy())
                all_logits.append(outputs.detach().cpu().numpy())

            all_labels = np.concatenate(all_labels, axis=0)
            all_logits = np.concatenate(all_logits, axis=0)

        return all_labels, all_logits

    test_probs = []
    for i in range(forward_passes):
        test_labels, test_logits = get_outputs_dropout(model, test_loader)
        test_probs.append(softmax(test_logits, axis=1))

    test_probs = np.array(test_probs)
    test_probs_mean, test_probs_std = test_probs.mean(0), test_probs.std(0)

    ece_mc = ECE(test_probs_mean, test_labels)
    ace_mc = BS(test_probs_mean, test_labels)

## Temperature scaling

In [ ]:
class TemperatureScaling():

    def __init__(self, temp=1, maxiter=50, solver="BFGS"):
        """
        Initialize class

        Params:
            temp (float): starting temperature, default 1
            maxiter (int): maximum iterations done by optimizer, however 8 iterations have been maximum.
        """
        self.temp = temp
        self.maxiter = maxiter
        self.solver = solver

    def _loss_fun(self, x, probs, true):
        # Calculates the loss using log-loss (cross-entropy loss)
        scaled_probs = self.predict(probs, x)
        loss = log_loss(y_true=true, y_pred=scaled_probs)
        return loss

    # Find the temperature
    def fit(self, logits, true):
        """
        Trains the model and finds optimal temperature

        Params:
            logits: the output from neural network for each class (shape [samples, classes])
            true: one-hot-encoding of true labels.

        Returns:
            the results of optimizer after minimizing is finished.
        """

        true = true.flatten()  # Flatten y_val
        opt = minimize(self._loss_fun, x0=1, args=(logits, true), options={'maxiter': self.maxiter}, method=self.solver)
        self.temp = opt.x[0]

        return opt

    def predict(self, logits, temp=None):
        """
        Scales logits based on the temperature and returns calibrated probabilities

        Params:
            logits: logits values of data (output from neural network) for each class (shape [samples, classes])
            temp: if not set use temperatures find by model or previously set.

        Returns:
            calibrated probabilities (nd.array with shape [samples, classes])
        """

        if not temp:
            return softmax(logits / self.temp, axis=1)
        else:
            return softmax(logits / temp, axis=1)

# input a calibration data
cali_logits = ...
cali_labels = ...

ts = TemperatureScaling()
ts.fit(cali_logits, cali_labels)
test_probs_ts = ts.predict(test_logits)